In [1]:
from functools import reduce

class Flower:
                
    def get_index(self, l):
        return reduce(lambda a, b: a * 3 + b, l, 0)
    
    def __init__(self, n, m, s, si):
        self.n = n
        self.m = m
        self.s = s
        self.si = si
        self.cm = {}
        
        for color, tl in m.items():
            for t in tl:
                assert len(t) == self.n
                for e in t:
                    assert e < 3
                index = self.get_index(t)
                assert index not in self.cm
                self.cm[self.get_index(t)] = color
                
        tc = 3 ** self.n
        ec = sum(map(lambda a: len(a), self.m.values()))
        assert tc == ec
        
        for ss in s:
            for e in ss:
                assert e < 3
            assert self.get_index(ss) in self.cm
    
    def get_tuple(self, l):
        r = [0] * self.n
        for i in range(self.n - 1, -1, -1):
            r[i] = l % 3
            l //= 3
        return r
        
    def info(self):
        return '{}\n{}\n{}\n{}\n{}'.format(self.n,  self.m, self.s, self.si, self.cm)
    
    def select_false_least_gen(self, m):
        i, g = None, None
        for k, v in m.items():
            [status, _, l] = v
            if status:
                continue
            for ll in l:
                eg = ll[0]
                if g is None or eg < g:
                    i = k
                    g = eg
        return i, g
    
    def mate(self, index1, index2):        
        def mate_inner(t1, t2, c, p, i):
            if i >= self.n:
                return [(self.get_index(c), p)]
            [n1, n2] = sorted([t1[i], t2[i]])
            r = []
            if n1 == 0:
                if n2 == 0:
                    c[i] = 0
                    r += mate_inner(t1, t2, c, p, i + 1)
                elif n2 == 1:
                    c[i] = 0
                    r += mate_inner(t1, t2, c, p / 2, i + 1)
                    c[i] = 1
                    r += mate_inner(t1, t2, c, p / 2, i + 1)
                elif n2 == 2:
                    c[i] = 1
                    r += mate_inner(t1, t2, c, p, i + 1)
            elif n1 == 1:
                if n2 == 1:
                    c[i] = 0
                    r += mate_inner(t1, t2, c, p / 4, i + 1)
                    c[i] = 1
                    r += mate_inner(t1, t2, c, p / 2, i + 1)
                    c[i] = 2
                    r += mate_inner(t1, t2, c, p / 4, i + 1)
                elif n2 == 2:
                    c[i] = 1
                    r += mate_inner(t1, t2, c, p / 2, i + 1)
                    c[i] = 2
                    r += mate_inner(t1, t2, c, p / 2, i + 1)
            elif n1 == 2:
                if n2 == 2:
                    c[i] = 2
                    r += mate_inner(t1, t2, c, p, i + 1)

            return r
        return mate_inner(self.get_tuple(index1), self.get_tuple(index2), [None] * self.n, 1.0, 0)
    
    def mate_definite(self, index1, index2):
        r = self.mate(index1, index2)
        m = {}
        results = []
        for t in r:
            i, p = t
            c = self.cm[i]
            if c not in m:
                m[c] = []
            m[c].append((i, p))
        for c, t in m.items():
            if len(t) == 1:
                results += t
        return results
        
    def definites(self):
        m = {}
        for i in range(0, 3 ** self.n):
            m[i] = [False, None, []]
        for s in self.s:
            i = self.get_index(s)
            m[i][2].append((0, None, None, 1.0))
        while True:
            i, g = self.select_false_least_gen(m)
            if i is None:
                break
            m[i][0] = True
            m[i][1] = g
            for ii, v in m.items():
                [status, gg, _] = v
                if not status:
                    continue
                a = self.mate_definite(i, ii)
                nextg = max(g, gg) + 1
                for ae in a:
                    (gt, p) = ae
                    m[gt][2].append((nextg, i, ii, p))
        filtered_items = filter(lambda a: a[1][0], m.items())
        sorted_tuples = sorted(map(lambda a: (a[1][1], a), filtered_items))
        gen_sorted = map(lambda a: a[1], sorted_tuples)
        def filter_self(l, ss):
            f = filter(lambda a: (a[1] != ss and a[2] != ss), l)
            return sorted(f)
        extracted = map(lambda a: (a[0], a[1][1], filter_self(a[1][2], a[0])), gen_sorted)
        return list(extracted)
    
    def print_definites(self):
        def get_g_color(i):
            return "{} ({})".format(self.cm[i], '-'.join(map(lambda a: str(a), self.get_tuple(i))))
        data = self.definites()
        for entry in data:
            (color, gen_number, binfos) = entry
            print(get_g_color(color) + " :")
            for binfo in binfos:
                [g, p1, p2, pp] = binfo
                if p1 is None or p2 is None:
                    print("\tSeed")
                else:
                    print("\t{}: {} x {} [{}%]".format(g, get_g_color(p1), get_g_color(p2), pp* 100))
            
    

In [2]:
windflower = Flower(3,{
    "Red": [
        (1,0,0),(1,0,1),
        (2,0,0),(2,0,1),(2,1,0),(2,1,1)],
    "Blue": [
        (1,0,2),
        (0,0,2),(0,1,2)],
    "Pink": [
        (1,1,0),(1,1,1),(1,1,2),
        (2,2,0),(2,2,1)],
    "Orange": [
        (1,2,0),(1,2,1),(1,2,2),
        (0,1,0),(0,1,1),(0,2,0),(0,2,1),(0,2,2)],
    "Purple": [
        (2,0,2),(2,1,2),(2,2,2)],
    "White": [
        (0,0,0),(0,0,1)]
    }, [
        (2,0,0),
        (0,2,0),
        (0,0,1)
    ], [
        (1,0,2),
        (2,2,1)
    ])
windflower.print_definites()

White (0-0-1) :
	Seed
	2: Pink (1-1-0) x Blue (0-0-2) [25.0%]
	3: White (0-0-0) x Blue (0-0-2) [100.0%]
	3: Orange (0-1-1) x Blue (0-0-2) [25.0%]
	3: Red (1-0-1) x Blue (0-0-2) [25.0%]
	3: Pink (1-1-1) x Blue (0-0-2) [12.5%]
	4: Orange (0-1-0) x Blue (0-0-2) [50.0%]
	4: Red (1-0-0) x Blue (0-0-2) [50.0%]
	5: Blue (1-0-2) x White (0-0-0) [50.0%]
	5: Blue (1-0-2) x Orange (0-1-0) [25.0%]
	5: Blue (1-0-2) x Orange (0-1-1) [12.5%]
	5: Blue (1-0-2) x Red (1-0-0) [25.0%]
	5: Blue (1-0-2) x Red (1-0-1) [12.5%]
	5: Blue (1-0-2) x Pink (1-1-0) [12.5%]
	5: Blue (1-0-2) x Pink (1-1-1) [6.25%]
Orange (0-2-0) :
	Seed
Red (2-0-0) :
	Seed
Blue (0-0-2) :
	1: White (0-0-1) x White (0-0-1) [25.0%]
Pink (1-1-0) :
	1: Red (2-0-0) x Orange (0-2-0) [100.0%]
	4: Orange (0-1-0) x Red (2-0-0) [50.0%]
	4: Red (1-0-0) x Orange (0-1-0) [25.0%]
	4: Red (1-0-0) x Orange (0-2-0) [50.0%]
White (0-0-0) :
	2: Pink (1-1-0) x Pink (1-1-0) [6.25%]
	4: Orange (0-1-0) x Orange (0-1-0) [25.0%]
	4: Orange (0-1-0) x Pink (1-1-

In [3]:
pansy = Flower(3,{
    "Red": [
        (1,0,0),(1,0,1),
        (2,0,0),(2,0,1),(2,1,0),(2,1,1)],
    "Blue": [
        (1,0,2),
        (0,0,2),(0,1,2)],
    "Orange": [
        (1,1,0),(1,1,1),(1,1,2),
        (2,2,0),(2,2,1)],
    "Yellow": [
        (1,2,0),(1,2,1),(1,2,2),
        (0,1,0),(0,1,1),(0,2,0),(0,2,1),(0,2,2)],
    "Purple": [
        (2,0,2),(2,1,2),(2,2,2)],
    "White": [
        (0,0,0),(0,0,1)]
    }, [
        (2,0,0),
        (0,2,0),
        (0,0,1)
    ], [
        (1,0,2),
        (2,2,1)
    ])
pansy.print_definites()

White (0-0-1) :
	Seed
	2: Orange (1-1-0) x Blue (0-0-2) [25.0%]
	3: White (0-0-0) x Blue (0-0-2) [100.0%]
	3: Yellow (0-1-1) x Blue (0-0-2) [25.0%]
	3: Red (1-0-1) x Blue (0-0-2) [25.0%]
	3: Orange (1-1-1) x Blue (0-0-2) [12.5%]
	4: Yellow (0-1-0) x Blue (0-0-2) [50.0%]
	4: Red (1-0-0) x Blue (0-0-2) [50.0%]
	5: Blue (1-0-2) x White (0-0-0) [50.0%]
	5: Blue (1-0-2) x Yellow (0-1-0) [25.0%]
	5: Blue (1-0-2) x Yellow (0-1-1) [12.5%]
	5: Blue (1-0-2) x Red (1-0-0) [25.0%]
	5: Blue (1-0-2) x Red (1-0-1) [12.5%]
	5: Blue (1-0-2) x Orange (1-1-0) [12.5%]
	5: Blue (1-0-2) x Orange (1-1-1) [6.25%]
Yellow (0-2-0) :
	Seed
Red (2-0-0) :
	Seed
Blue (0-0-2) :
	1: White (0-0-1) x White (0-0-1) [25.0%]
Orange (1-1-0) :
	1: Red (2-0-0) x Yellow (0-2-0) [100.0%]
	4: Yellow (0-1-0) x Red (2-0-0) [50.0%]
	4: Red (1-0-0) x Yellow (0-1-0) [25.0%]
	4: Red (1-0-0) x Yellow (0-2-0) [50.0%]
White (0-0-0) :
	2: Orange (1-1-0) x Orange (1-1-0) [6.25%]
	4: Yellow (0-1-0) x Yellow (0-1-0) [25.0%]
	4: Yellow (0-1-0

In [4]:
cosmos = Flower(3,{
    "Pink": [
        (1,0,0),(1,0,1),(1,0,2),(1,1,2)],
    "Orange": [
        (1,1,0),(1,1,1),(1,2,0),(1,2,1),(1,2,2),
        (2,1,0),(2,1,1)],
    "Red": [
        (2,0,0),(2,0,1),(2,0,2),(2,1,2),(2,2,2)],
    "Black": [
        (2,2,0),(2,2,1)],
    "Yellow": [
        (0,1,0),(0,1,1),(0,2,0),(0,2,1),(0,2,2)],
    "White": [
        (0,0,0),(0,0,1),(0,0,2),(0,1,2)]
    }, [
        (2,0,0),
        (0,2,1),
        (0,0,1)
    ], [
        (1,1,2),
        (2,1,1)
    ])
cosmos.print_definites()

White (0-0-1) :
	Seed
Yellow (0-2-1) :
	Seed
Red (2-0-0) :
	Seed
White (0-1-2) :
	1: Yellow (0-2-1) x White (0-0-1) [25.0%]
	3: Yellow (0-1-1) x Yellow (0-2-1) [12.5%]
	3: Yellow (0-2-2) x White (0-0-1) [50.0%]
	3: Yellow (0-2-2) x Yellow (0-1-1) [25.0%]
	3: Pink (1-0-1) x Yellow (0-2-1) [12.5%]
	3: Pink (1-0-1) x Yellow (0-2-2) [25.0%]
	3: Orange (1-1-1) x Yellow (0-2-1) [6.25%]
	3: Orange (1-1-1) x Yellow (0-2-2) [12.5%]
	4: Pink (1-1-2) x Yellow (0-2-1) [12.5%]
	4: Pink (1-1-2) x Yellow (0-2-2) [25.0%]
	5: Orange (1-2-2) x White (0-0-1) [25.0%]
	5: Orange (1-2-2) x Yellow (0-1-1) [12.5%]
	5: Orange (1-2-2) x Pink (1-0-1) [12.5%]
	5: Orange (1-2-2) x Orange (1-1-1) [6.25%]
	5: Orange (1-2-2) x Pink (1-1-2) [12.5%]
Yellow (0-1-1) :
	2: White (0-1-2) x White (0-0-1) [25.0%]
	3: Yellow (0-2-2) x White (0-0-1) [50.0%]
	3: Pink (1-0-1) x White (0-1-2) [12.5%]
	3: Pink (1-0-1) x Yellow (0-2-2) [25.0%]
	4: Pink (1-1-2) x White (0-0-1) [12.5%]
	4: Pink (1-1-2) x Pink (1-0-1) [6.25%]
	5: Oran

In [5]:
tulip = Flower(3,{
    "Red": [
        (1,0,0),
        (2,0,1),(2,0,2),(2,1,1),(2,1,2)],
    "Pink": [
        (1,0,1)],
    "White": [
        (1,0,2),
        (0,0,0),(0,0,1),(0,0,2),(0,1,2)],
    "Orange": [
        (1,1,0),(1,2,0)],
    "Yellow": [
        (1,1,1),(1,1,2),(1,2,1),(1,2,2),
        (0,1,0),(0,1,1),(0,2,0),(0,2,1),(0,2,2)],
    "Black": [
        (2,0,0),(2,1,0)],
    "Purple": [
        (2,2,0),(2,2,1),(2,2,2)]
    }, [
        (2,0,1),
        (0,2,0),
        (0,0,1)
    ], [
        (1,0,1),
        (1,2,0),
        (2,1,0)
    ])
tulip.print_definites()

White (0-0-1) :
	Seed
	2: White (1-0-2) x Red (1-0-0) [25.0%]
	2: Orange (1-1-0) x White (1-0-2) [12.5%]
	3: White (0-0-0) x White (1-0-2) [50.0%]
	3: Yellow (0-1-0) x White (1-0-2) [25.0%]
Yellow (0-2-0) :
	Seed
	4: Orange (1-2-0) x Orange (1-2-0) [25.0%]
Red (2-0-1) :
	Seed
	2: White (1-0-2) x Red (1-0-0) [25.0%]
	2: Black (2-0-0) x White (1-0-2) [50.0%]
	3: Red (2-0-2) x Red (1-0-0) [50.0%]
	3: Red (2-0-2) x Black (2-0-0) [100.0%]
Red (1-0-0) :
	1: Red (2-0-1) x White (0-0-1) [25.0%]
	2: Pink (1-0-1) x White (0-0-1) [12.5%]
	2: Orange (1-1-0) x White (0-0-1) [12.5%]
	2: Orange (1-1-0) x Orange (1-1-0) [12.5%]
	2: Yellow (1-1-1) x White (0-0-1) [6.25%]
	2: Black (2-0-0) x White (0-0-1) [50.0%]
	2: Black (2-0-0) x Orange (1-1-0) [25.0%]
	3: White (0-0-0) x Pink (1-0-1) [25.0%]
	3: White (0-0-0) x Orange (1-1-0) [25.0%]
	3: White (0-0-0) x Yellow (1-1-1) [12.5%]
	3: White (0-0-0) x Black (2-0-0) [100.0%]
	3: White (0-0-0) x Red (2-0-1) [50.0%]
	3: Yellow (0-1-0) x Pink (1-0-1) [12.5%]


In [6]:
mums = Flower(3,{
    "Pink": [
        (1,0,0),(1,0,1),(1,0,2),
        (1,1,2)],
    "Yellow": [
        (1,1,0),
        (0,1,0),(0,1,1),(0,2,0),(0,2,1),(0,2,2)],
    "Red": [
        (1,1,1),
        (2,0,0),(2,0,1),(2,0,2),(2,1,2),(2,2,2)],
    "Purple": [
        (1,2,0),(1,2,1),(1,2,2),
        (2,1,0),(2,1,1),
        (0,0,2)],
    "Green": [
        (2,2,0),(2,2,1)],
    "White": [
        (0,0,0),(0,0,1),(0,1,2)]
    }, [
        (2,0,0),
        (0,2,0),
        (0,0,1)
    ], [
        (1,1,2),
        (2,1,1)
    ])
mums.print_definites()

White (0-0-1) :
	Seed
	2: Yellow (1-1-0) x Purple (0-0-2) [25.0%]
	3: White (0-0-0) x Purple (0-0-2) [100.0%]
	3: Pink (1-0-0) x Purple (0-0-2) [50.0%]
	3: Pink (1-0-1) x Purple (0-0-2) [25.0%]
	4: Yellow (0-1-0) x Purple (0-0-2) [50.0%]
Yellow (0-2-0) :
	Seed
	3: Purple (1-2-0) x Purple (1-2-0) [25.0%]
Red (2-0-0) :
	Seed
	2: Yellow (1-1-0) x Yellow (1-1-0) [6.25%]
	3: Pink (1-0-0) x Pink (1-0-0) [25.0%]
	3: Pink (1-0-0) x Yellow (1-1-0) [12.5%]
	3: Purple (2-1-0) x Pink (1-0-0) [25.0%]
	3: Purple (2-1-0) x Yellow (1-1-0) [12.5%]
	3: Purple (2-1-0) x Purple (2-1-0) [25.0%]
Purple (0-0-2) :
	1: White (0-0-1) x White (0-0-1) [25.0%]
	3: Yellow (0-1-1) x White (0-0-1) [12.5%]
	3: Yellow (0-1-1) x Yellow (0-1-1) [6.25%]
	3: Pink (1-0-1) x White (0-0-1) [12.5%]
	3: Pink (1-0-1) x Yellow (0-1-1) [6.25%]
	3: Pink (1-0-1) x Pink (1-0-1) [6.25%]
	3: Red (1-1-1) x White (0-0-1) [6.25%]
Yellow (1-1-0) :
	1: Red (2-0-0) x Yellow (0-2-0) [100.0%]
	3: Yellow (0-1-1) x Red (2-0-0) [25.0%]
	3: Red (1

In [7]:
lily = Flower(3,{
    "Pink": [
        (1,0,1),
        (2,0,2),(2,1,2)],
    "Yellow": [
        (1,1,1),(1,1,2),(1,2,1),(1,2,2),
        (0,1,0),(0,2,0),(0,2,1)],
    "Red": [
        (1,0,0),
        (2,0,1),(2,1,1)],
    "Orange": [
        (1,1,0),(1,2,0),
        (2,2,0),(2,2,1)],
    "Black": [
        (2,0,0),(2,1,0)],
    "White": [
        (0,0,0),(0,0,1),(0,0,2),(0,1,1),(0,1,2),(0,2,2),
        (1,0,2),
        (2,2,2)]
    }, [
        (2,0,1),
        (0,2,0),
        (0,0,2)
    ], [
        (2,2,1),
        (2,1,0),
        (2,1,2)
    ])
lily.print_definites()

White (0-0-2) :
	Seed
Yellow (0-2-0) :
	Seed
Red (2-0-1) :
	Seed
	2: White (1-0-2) x Pink (1-0-1) [12.5%]
	2: Black (2-0-0) x White (1-0-2) [50.0%]
	2: Pink (2-0-2) x Pink (1-0-1) [25.0%]
	2: Pink (2-0-2) x Black (2-0-0) [100.0%]
	3: Red (1-0-0) x White (1-0-2) [25.0%]
	3: Red (1-0-0) x Pink (2-0-2) [50.0%]
White (0-1-1) :
	1: Yellow (0-2-0) x White (0-0-2) [100.0%]
	2: Pink (1-0-1) x Yellow (0-2-0) [25.0%]
	2: White (1-0-2) x Yellow (0-2-0) [50.0%]
	2: Yellow (1-1-1) x Yellow (0-2-0) [12.5%]
	4: White (0-0-1) x Yellow (0-2-0) [50.0%]
Pink (1-0-1) :
	1: Red (2-0-1) x White (0-0-2) [50.0%]
	2: White (0-1-1) x Red (2-0-1) [25.0%]
	2: White (1-0-2) x White (0-1-1) [12.5%]
	2: Orange (1-1-0) x White (0-0-2) [25.0%]
	2: Orange (1-1-0) x White (0-1-1) [6.25%]
	2: Orange (1-1-0) x White (1-0-2) [25.0%]
	2: Orange (1-1-0) x Red (2-0-1) [12.5%]
	2: Yellow (1-1-1) x White (0-0-2) [12.5%]
	2: Yellow (1-1-1) x White (0-1-1) [6.25%]
	2: Yellow (1-1-1) x Orange (1-1-0) [6.25%]
	2: Black (2-0-0) x Wh

In [8]:
hyacinth = Flower(3,{
    "Pink": [
        (1,0,1)],
    "Yellow": [
        (1,1,1),(1,1,2),(1,2,1),(1,2,2),
        (0,1,0),(0,1,1),(0,2,0),(0,2,1),(0,2,2)],
    "Red": [
        (1,0,0),
        (2,0,0),(2,0,1),(2,0,2),(2,1,1),(2,1,2)],
    "Orange": [
        (1,1,0),(1,2,0)],
    "Purple": [
        (2,2,0),(2,2,1),(2,2,2)],
    "Blue": [
        (0,0,2),
        (2,1,0)],
    "White": [
        (0,0,0),(0,0,1),(0,1,2),
        (1,0,2)]
    }, [
        (2,0,1),
        (0,2,0),
        (0,0,1)
    ], [
        (1,0,1),
        (2,1,0)
    ])
lily.print_definites()

White (0-0-2) :
	Seed
Yellow (0-2-0) :
	Seed
Red (2-0-1) :
	Seed
	2: White (1-0-2) x Pink (1-0-1) [12.5%]
	2: Black (2-0-0) x White (1-0-2) [50.0%]
	2: Pink (2-0-2) x Pink (1-0-1) [25.0%]
	2: Pink (2-0-2) x Black (2-0-0) [100.0%]
	3: Red (1-0-0) x White (1-0-2) [25.0%]
	3: Red (1-0-0) x Pink (2-0-2) [50.0%]
White (0-1-1) :
	1: Yellow (0-2-0) x White (0-0-2) [100.0%]
	2: Pink (1-0-1) x Yellow (0-2-0) [25.0%]
	2: White (1-0-2) x Yellow (0-2-0) [50.0%]
	2: Yellow (1-1-1) x Yellow (0-2-0) [12.5%]
	4: White (0-0-1) x Yellow (0-2-0) [50.0%]
Pink (1-0-1) :
	1: Red (2-0-1) x White (0-0-2) [50.0%]
	2: White (0-1-1) x Red (2-0-1) [25.0%]
	2: White (1-0-2) x White (0-1-1) [12.5%]
	2: Orange (1-1-0) x White (0-0-2) [25.0%]
	2: Orange (1-1-0) x White (0-1-1) [6.25%]
	2: Orange (1-1-0) x White (1-0-2) [25.0%]
	2: Orange (1-1-0) x Red (2-0-1) [12.5%]
	2: Yellow (1-1-1) x White (0-0-2) [12.5%]
	2: Yellow (1-1-1) x White (0-1-1) [6.25%]
	2: Yellow (1-1-1) x Orange (1-1-0) [6.25%]
	2: Black (2-0-0) x Wh

In [9]:
rose = Flower(4,{
    "Red": [
        (1,0,0,0),(1,0,1,0),(1,0,2,0),
        (1,1,1,0),(1,1,2,0),
        (1,2,2,0),
        (2,0,0,1),(2,0,1,1),(2,0,2,1),
        (2,1,1,0),(2,1,1,1),(2,1,2,1),
        (2,2,2,1)
    ],
    "Pink": [
        (1,0,0,1),(1,0,1,1),(1,0,2,1),
        (1,1,1,1),(1,1,2,1),
        (1,2,2,1),
        (2,0,0,2),(2,0,1,2),(2,0,2,2)
    ],
    "White": [
        (0,0,0,0),(0,0,0,1),(0,0,0,2),(0,0,1,0),(0,0,1,1),(0,0,1,2),
        (0,1,1,0),(0,1,1,1),(0,1,1,2),
        (0,2,2,0),(0,2,2,1),(0,2,2,2),
        (1,0,0,2),(1,0,1,2),
        (1,1,1,2),
        (1,2,2,2),
        (2,1,1,2),
        (2,2,2,2)
    ],
    "Purple":[
        (0,0,2,0),(0,0,2,1),(0,0,2,2),
        (0,1,2,0),(0,1,2,1),(0,1,2,2),
        (1,0,2,2),
        (1,1,2,2),
        (2,1,2,2)
    ],
    "Orange": [
        (1,1,0,0),
        (1,2,0,0),(1,2,1,0),
        (2,1,0,0),(2,1,0,1),
        (2,2,0,0),(2,2,0,1),(2,2,1,0),(2,2,1,1)
    ],
    "Yellow": [
        (0,1,0,0),(0,1,0,1),(0,1,0,2),
        (0,2,0,0),(0,2,0,1),(0,2,0,2),(0,2,1,0),(0,2,1,1),(0,2,1,2),
        (1,1,0,1),(1,1,0,2),
        (1,2,0,1),(1,2,0,2),(1,2,1,1),(1,2,1,2),
        (2,1,0,2),
        (2,2,0,2),(2,2,1,2)
    ],
    "Black": [
        (2,0,0,0),(2,0,1,0),(2,0,2,0),
        (2,1,2,0)
    ],
    "Blue": [
        (2,2,2,0)
    ]}, [
        (2,0,0,1),
        (0,0,1,0),
        (0,2,0,0)
    ], [
        (2,0,2,2),
        (2,2,1,1)
    ])
rose.print_definites()

White (0-0-1-0) :
	Seed
	3: White (0-0-0-0) x Purple (0-0-2-0) [100.0%]
	3: Red (1-0-0-0) x Purple (0-0-2-0) [50.0%]
	3: Red (1-0-1-0) x Purple (0-0-2-0) [25.0%]
	10: Red (1-0-2-0) x White (0-0-0-0) [50.0%]
	10: Red (1-0-2-0) x Red (1-0-0-0) [25.0%]
	10: Red (1-0-2-0) x Red (1-0-1-0) [12.5%]
Yellow (0-2-0-0) :
	Seed
Red (2-0-0-1) :
	Seed
	2: Pink (2-0-0-2) x Orange (1-1-0-0) [25.0%]
	2: Pink (2-0-0-2) x Yellow (1-1-0-1) [12.5%]
	2: Pink (2-0-0-2) x Black (2-0-0-0) [100.0%]
	3: Red (1-0-0-0) x Pink (2-0-0-2) [50.0%]
	3: Pink (1-0-0-1) x Pink (2-0-0-2) [25.0%]
	3: White (1-0-0-2) x Red (1-0-0-0) [25.0%]
	3: White (1-0-0-2) x Pink (1-0-0-1) [12.5%]
	3: White (1-0-0-2) x Orange (1-1-0-0) [12.5%]
	3: White (1-0-0-2) x Yellow (1-1-0-1) [6.25%]
	3: White (1-0-0-2) x Black (2-0-0-0) [50.0%]
	3: Orange (2-1-0-1) x White (1-0-0-2) [12.5%]
	3: Orange (2-1-0-1) x Black (2-0-0-0) [25.0%]
	3: Orange (2-1-0-1) x Pink (2-0-0-2) [25.0%]
	3: Orange (2-1-0-1) x Orange (2-1-0-1) [12.5%]
	4: Yellow (2-1-0-